<a href="https://colab.research.google.com/github/it-math/django_week1/blob/master/hh_res.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [122]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import re

In [123]:
df = pd.read_csv("https://it-math.ru/wp-content/uploads/it_vacancies.csv")

In [ ]:
type(df)

In [ ]:
df.info()

In [126]:
df = df.drop('key_skills',axis=1)

In [ ]:
df.info()

In [128]:
TOP_N = 20  # Сколько городов показать

In [185]:
CATEGORIES = {
    "backend": ["python", "django", "flask", "java", "spring", "php", "node.js", "ruby"],
    "frontend": ["javascript", "react", "vue", "angular", "html", "css", "typescript"],
    "data_science": ["data scientist", "machine learning", "ml", "pandas", "numpy", "ai", "анализ данных", 'ds'],
    "qa": ["тестировщик", "qa", "ручное тестирование", "автотесты", "selenium", "postman"],
    "devops": ["devops", "docker", "kubernetes", "ci/cd", "linux", "ansible", "aws", "azure"],
    "mobile": ["android", "ios", "flutter", "react native", "swift", "kotlin"],
    "management": ["менеджер", "руководитель проекта", "product manager", "scrum", "jira"],
    "programmer" : []
}

In [ ]:
df["avg_salary"] = df[["salary_from", "salary_to"]].mean(axis=1)
# df["avg_salary"] = df[["salary_from", "salary_to"]].dropna().mean(axis=1)
#df["avg_salary"] = df[["salary_from", "salary_to"]].mean(axis=1, skipna=False)
df[["name", "salary_from", "salary_to", "avg_salary"]]

**Подсчёт количества вакансий по городам**

In [ ]:
city_counts = df["city"].value_counts().head(TOP_N)
print(city_counts)

In [ ]:
# --- График: столбчатая диаграмма ---
plt.figure(figsize=(12, 6))
bars = plt.bar(city_counts.index, city_counts.values, color="#4F94CD")
plt.title("Количество IT-вакансий по городам")
plt.xlabel("Город")
plt.ylabel("Число вакансий")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()

# Подписи значений над столбцами
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval + 0.5,
             str(int(yval)), ha='center', va='bottom', fontsize=10)

plt.show()

In [ ]:
# --- Круговая диаграмма ---
plt.figure(figsize=(10, 10))
city_counts.plot.pie(
    autopct='%1.1f%%',
    startangle=90,
    colors=plt.cm.Paired.colors,
    textprops={'fontsize': 8},
    title=f'Распределение вакансий по {TOP_N} городам'
)
plt.ylabel('')
plt.tight_layout()
plt.show()

**Распределение требуемого опыта**

In [ ]:
experience_counts = df["experience"].value_counts()
print("Распределение требуемого опыта:")
print(experience_counts)

# Круговая диаграмма
experience_counts.plot(kind="pie", autopct='%1.1f%%', title="Требуемый опыт в вакансиях")
plt.ylabel("")
plt.tight_layout()
plt.show()

**Средняя зарплата и опыт**

In [197]:
df.groupby("experience")[["avg_salary"]].agg(["count", "mean"])

avg_salary               
                        count           mean
experience                                  
Более 6 лет               116  240373.275862
Нет опыта                 428   71248.489486
От 1 года до 3 лет       1013  114017.747285
От 3 до 6 лет             603  194801.143449

**Классификация по опыту**

In [198]:
def classify_seniority(name):
    name = name.lower()
    if "senior" in name or "ведущий" in name or "lead" in name:
        return "Senior"
    elif "junior" in name or "стажер" in name or "intern" in name:
        return "Junior"
    else:
        return "Middle+"

df["seniority"] = df["name"].apply(classify_seniority)
df.groupby("seniority")["name"].count()

,name
seniority,
Junior,325
Middle+,3813
Senior,512


**Есть ли связь между уровнем зарплаты и seniority?**

In [ ]:
salary_by_seniority = df.groupby("seniority")["avg_salary"].mean().sort_values(ascending=False)
print("Средняя зарплата по уровню (Junior/Middle/Senior):")
print(salary_by_seniority)

# График
salary_by_seniority.plot(kind="bar", color="teal", title="Средняя зарплата по уровню")
plt.ylabel("Средняя зарплата")
plt.xlabel("Уровень")
plt.tight_layout()
plt.show()

In [ ]:
df['prof'] = df['prof_role'].apply(lambda x: x[2:-2])
# Получаем уникальные элементы
unique_elements = sorted(df['prof'].dropna().unique())
print(unique_elements)

In [ ]:
df['prof']

In [ ]:
prof_counts = df["prof"].value_counts().head(TOP_N)
print(prof_counts)

**Средние зарплаты по профессиям**

In [ ]:
salary_by_prof = df.groupby("prof")["avg_salary"].mean().sort_values(ascending=False)
print(salary_by_prof)

**Средние зарплаты по категориям:**

In [140]:
def categorize_vacancy(row):
    text = f"{row['name']} {row['requirement']}".lower() if pd.notna(row['requirement']) else row['name'].lower()

    for category, keywords in CATEGORIES.items():
        for keyword in keywords:
            if re.search(rf"\b{re.escape(keyword.lower())}\b", text, re.IGNORECASE):
                return category
    return "other"

df["category"] = df.apply(categorize_vacancy, axis=1)

In [ ]:
salary_by_category = df.groupby("category")["avg_salary"].mean().sort_values(ascending=False)
print(salary_by_category)

**График зарплат по категориям**

In [ ]:
#
plt.figure(figsize=(10, 6))
sns.barplot(x=salary_by_category.values, y=salary_by_category.index, palette="viridis")
plt.title("Средняя зарплата по категориям")
plt.xlabel("Средняя зарплата (руб.)")
plt.ylabel("Категория")
plt.grid(True, axis="x", linestyle="--", alpha=0.5)
plt.tight_layout()
plt.show()

**Подсчёт количества вакансий по городам**

In [ ]:
city_counts = df["city"].value_counts().head(TOP_N)
print(city_counts)

In [ ]:
# --- График: столбчатая диаграмма ---
plt.figure(figsize=(12, 6))
bars = plt.bar(city_counts.index, city_counts.values, color="#4F94CD")
plt.title("Количество IT-вакансий по городам")
plt.xlabel("Город")
plt.ylabel("Число вакансий")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()

# Подписи значений над столбцами
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval + 0.5,
             str(int(yval)), ha='center', va='bottom', fontsize=10)

plt.show()

In [ ]:
# --- Круговая диаграмма ---
plt.figure(figsize=(10, 10))
city_counts.plot.pie(
    autopct='%1.1f%%',
    startangle=90,
    colors=plt.cm.Paired.colors,
    textprops={'fontsize': 8},
    title=f'Распределение вакансий по {TOP_N} городам'
)
plt.ylabel('')
plt.tight_layout()
plt.show()

**Бокс-плот зарплат по категориям**

In [ ]:
# Фильтрация: убираем пустые значения
df_filtered = df.dropna(subset=["avg_salary", "category"])

# Уберём нереалистично высокие зарплаты (например, свыше 500 000)
df_filtered = df_filtered[df_filtered["avg_salary"] < 500000]

# Сортировка по средней зарплате внутри категорий
category_order = df_filtered.groupby("category")["avg_salary"].median().sort_values(ascending=False).index

# Бокс-плот
plt.figure(figsize=(12, 8))
sns.boxplot(
    data=df_filtered,
    x="avg_salary",
    y="category",
    order=category_order,
    palette="viridis"
)

plt.title("Распределение зарплат по категориям (boxplot)")
plt.xlabel("Средняя зарплата (руб.)")
plt.ylabel("Категория")
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

**Бокс-плот по городам**

In [ ]:
# Фильтрация по городам
top_cities = df["city"].value_counts().head(10).index
df_city_filtered = df_filtered[df_filtered["city"].isin(top_cities)]

# Бокс-плот по городам
plt.figure(figsize=(12, 8))
sns.boxplot(
    data=df_city_filtered,
    x="avg_salary",
    y="city",
    order=df_city_filtered.groupby("city")["avg_salary"].median().sort_values(ascending=False).index,
    palette="coolwarm"
)

plt.title("Распределение зарплат по городам (boxplot)")
plt.xlabel("Средняя зарплата (руб.)")
plt.ylabel("Город")
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

**Самые популярные навыки**

In [ ]:
all_skills = df["requirement"].str.split(",").explode().str.strip()
top_skills = all_skills.value_counts().head(30)
print(top_skills)

In [ ]:
top_skills.plot(kind="barh", title="ТОП-20 ключевых навыков")
plt.gca().invert_yaxis()
plt.xlabel("Частота")
plt.ylabel("Навык")
plt.tight_layout()
plt.show()

**Самые популярные обязанности**

In [ ]:
all_respons = df["responsibility"].str.split(", ").explode().str.strip()
all_respons.value_counts().head(20).plot(kind="barh", title="ТОП-20 ключевых обязанностей")
plt.gca().invert_yaxis()
plt.xlabel("Частота")
plt.ylabel("Обязанность")
plt.tight_layout()
plt.show()

In [149]:
combined = pd.concat([all_skills, all_respons], ignore_index=True)

In [ ]:
combined.value_counts().head(20).plot(kind="barh", title="ТОП-20 ключевых обязанностей")
plt.gca().invert_yaxis()
plt.xlabel("Частота")
plt.ylabel("Навыки и обязанности")
plt.tight_layout()
plt.show()

**Облако навыков**

In [154]:
def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r"[^\w\s]", " ", text)  # убираем знаки препинания
    text = re.sub(r"\d+", " ", text)      # убираем цифры
    text = re.sub(r"\s+", " ", text).strip()
    return text

In [158]:
# @title Стоп-слова
# --- Список стоп-слов ---
STOPWORDS_RU = {
    "и", "в", "на", "с", "по", "для", "из", "от", "до", "за", "над", "под", "при",
    "как", "лет", "опыта","менее", "х", "опыта", "тебя",
    "к", "у", "о", "об", "со", "а", "но", "или", "да", "же", "то", "это", "он", "она",
    "более", "работы", "года", "они", "мы", "вы", "они", "его", "её", "их", "наш", "ваш", "этот", "тот", "такой",
    "мой", "твой", "свой", "быть", "был", "была", "были", "есть", "нет", "да", "не", "ни", "будет", "преимуществом"
}
STOPWORDS_RU_COMMON = {
    "и", "в", "на", "с", "по", "для", "из", "от", "до", "за", "над", "под", "при",
    "к", "у", "о", "об", "со", "а", "но", "или", "да", "же", "то", "это", "он", "она",
    "они", "мы", "вы", "их", "мой", "твой", "свой", "наш", "ваш", "этот", "тот", "такой",
    "быть", "был", "была", "были", "есть", "нет", "да", "не", "ни", "уже", "ещё", "бы", "либо", "до",
    "всё", "всё-таки", "весь", "все", "всех", "всем", "всему", "всего", "всеми", "том числе", "быстро", "выше"
}

STOPWORDS_JOB_EXTRA = {
    "работа", "вакансия", "компания", "уровень", "зарплата", "оклад", "готовность","знакомство",
    "график", "полный день", "частично", "удалёнка", "офис", "стажировка", "стажер", "включая",
    "junior", "мидл", "senior", "lead", "руководитель", "специалист", "опыт", "отличное", "организации",
    "знание", "умение", "умения", "навыки", "желательно", "обязательно", "требуется", "плюсом",
    "развитие", "карьера", "возможности", "программа", "трудоустройство", "перспективы",
    "интересный проект", "динамичная компания", "команда", "работать", "место", "роль", "минимум",
    "должность", "условия", "предоставляется", "предоставляем", "предлагается", "предлагаем",
    "вам", "вас", "вам нужно", "мы предлагаем", "мы ищем", "мы ищем вас", "вы будете", "проведения",
    "что", "чтобы", "бы", "можно", "должен", "должны", "требование", "требования", "ответственность",
    "обязанности", "уровень знаний", "опыт работы", "опыт", "желателен", "желательно", "понимание",
    "владение", "владеете", "готовы", "готов", "готовность", "работать", "работе", "работаю", "умеете",
    "мы", "наша", "наши", "вам", "ваш", "ваша", "ваши", "все", "всё", "всегда", "всё равно", "знаете",
    "каждый", "любой", "любая", "любое", "кто", "что", "как", "почему", "сколько", "где", "имеете", "имеешь",
    "также", "в том числе", "например", "например:", "и т. д.", "итд", "итп", "и другие", "приветствуется", "написания",
    "и прочее", "и пр.", "другие", "прочее", "другое", "и так далее", "и тому подобное", "имеет", "хорошее", "систем", "наличие"
}

STOPWORDS_TECH_EXTRA = {
    "разработка", "разрабатывать", "работать", "работа", "система", "создание","приложений", "аналогичной должности",
    "веб", "интернет", "программа", "проект", "решение", "платформа", "сервис", "инструмент", "сфере",
    "технология", "библиотека", "метод", "модель", "данные", "информация", "задача", "задачи", "разработки", "разработке",
    "функция", "код", "писать", "написание", "тестировать", "пользователь", "клиент", "заказчик", "настройки", "использования", "знаешь",
    "системный", "системы", "хранение", "структура", "требование", "требовать", "использования", "технологий", "системами",
    "анализировать", "развитие", "развивать", "инфраструктура", "масштабируемость", "принципов", "кода", "языка", "коммерческой",
    "производительность", "оптимизация", "скорость", "взаимодействие", "интеграция", "управления", "процессов", "разработчика"
}

STOPWORDS = STOPWORDS_RU_COMMON.union(STOPWORDS_JOB_EXTRA).union(STOPWORDS_TECH_EXTRA).union(STOPWORDS_RU)

In [ ]:
all_skills = " ".join(df["requirement"].fillna("").apply(clean_text))

#filtered_skills = " ".join([word for word in all_skills.split() if word not in STOPWORDS])
#all_skills = filtered_skills

# --- Создание облака слов ---
wordcloud = WordCloud(
    width=800,
    height=400,
    background_color="white",
    max_words=200,
    contour_width=3,
    contour_color='steelblue',
    colormap="viridis"
).generate(all_skills)

# Облако навыков
plt.figure(figsize=(12, 6))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.title("Часто встречающиеся ключевые навыки", fontsize=14)
plt.tight_layout()
plt.show()

**Распределение требования к опыту по категориям**

In [ ]:
df.groupby(["seniority", "category"])["name"].count().unstack(fill_value=0)

**Категории с количеством вакансий и зарплатами**

In [ ]:
report = df.groupby("category").agg(
    vacancies=("name", "count"),
    avg_salary=("avg_salary", "mean"),
    min_salary=("salary_from", "min"),
    max_salary=("salary_to", "max"),
    companies=("company", "nunique")
).sort_values(by="vacancies", ascending=False)

print(report)

**Компании и количество вакансий**

In [ ]:
df["company"].value_counts().head(20)

**Вакансии компаний по категориям**

In [ ]:
# Группируем по компаниям и категориям
company_category = df.groupby(["company", "category"]).size().unstack(fill_value=0)

# Считаем общее число вакансий у каждой компании
company_category["total"] = company_category.sum(axis=1)

# Сортируем по убыванию
company_category = company_category.sort_values(by="total", ascending=False)

# Удаляем вспомогательный столбец
company_category = company_category.drop(columns=["total"])

print(company_category.head(20))

**Топ-5 компаний по каждой категории**

In [ ]:
top_companies_per_category = {}

for category in df["category"].unique():
    top_companies = df[df["category"] == category].groupby("company").size().sort_values(ascending=False).head(5)
    top_companies_per_category[category] = top_companies

# Вывод
for category, companies in top_companies_per_category.items():
    print(f"Топ-5 компаний в категории '{category}':")
    print(companies,'\n')

**Тепловая карта**

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Подготовим данные
company_category_top = company_category.head(20)  # ограничимся 20 компаниями для наглядности

plt.figure(figsize=(14, 8))
sns.heatmap(company_category_top, annot=True, fmt="d", cmap="YlGnBu", linewidths=.5)
plt.ylabel("Компания")
plt.xlabel("Категория")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

**Самые "зарплатные" компании**

In [ ]:
top_salary_companies = df.groupby("company")["avg_salary"].mean().sort_values(ascending=False).head(30)
print(top_salary_companies)

**Самые "зарплатные" города**

In [ ]:
top_cities = df.groupby("city")["avg_salary"].mean().sort_values(ascending=False).head(10)
print(top_cities)

**Как часто встречается удалённая работа?**

In [ ]:
remote_count = df[df["requirement"].str.contains("удален", case=False, na=False)]
print(f"Вакансий с возможностью удалённой работы: {len(remote_count)}")

# По категориям
remote_by_category = remote_count["category"].value_counts()
print("Удалёнка по категориям:")
print(remote_by_category)

**Сравнение зарплат по опыту**

In [ ]:
salary_by_exp = df.groupby("experience")["avg_salary"].mean().sort_values(ascending=False)
print(salary_by_exp)

In [ ]:
salary_by_exp.plot(kind="bar", title="Средняя зарплата по опыту")
plt.ylabel("Средняя зарплата")
plt.xlabel("Требуемый опыт")
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

**Самые популярные профессии по поисковым запросам**

In [ ]:
pivot = pd.pivot_table(df, index="search_term", columns="category", aggfunc="size", fill_value=0)
print(pivot.head(20))